In [ ]:
!git clone https://github.com/IntelLabs/distiller.git
%cd distiller
!pip3 install -e .

fatal: destination path 'distiller' already exists and is not an empty directory.
/content/distiller
Obtaining file:///content/distiller
     |████████████████████████████████| 2.1MB 8.8MB/s 
     |████████████████████████████████| 734.6MB 24kB/s 
     |████████████████████████████████| 10.2MB 44.5MB/s 
     |████████████████████████████████| 460kB 53.5MB/s 
     |████████████████████████████████| 110.5MB 32kB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 798kB 46.8MB/s 
     |████████████████████████████████| 112kB 50.1MB/s 
     |████████████████████████████████| 1.7MB 45.4MB/s 
     |████████████████████████████████| 235kB 47.6MB/s 
     |████████████████████████████████| 153kB 51.7MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 6.7MB 21.3MB/s 
     |████████████████████████████████| 1.5MB 25.7MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |██████████████████████

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import time
import math
from tqdm.autonotebook import tqdm

import distiller
import distiller.models
from collections import OrderedDict

ModuleNotFoundError: ignored

In [ ]:
transformTrain = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transformTest = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = transformTrain)

trainLoader = torch.utils.data.DataLoader(trainset, batch_size = 128,
                                          shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform = transformTest)
testLoader = torch.utils.data.DataLoader(test, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#CIFAR10 does not have pretrained models
net = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
name = "resnet-56-distiller"
epochs = 200

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.backends.cudnn.benchmark = True
net = net.to(device)
print(device)

cuda


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), momentum = 0.9, weight_decay = 1e-4, nesterov=True, lr=0.3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=45, gamma=0.1)
checkpoint = {}
#distiller epochs start from 0
checkpoint["arch"] = "resnet56_cifar"
checkpoint["is_parallel"] = True 
checkpoint["dataset"] = "cifar10" 
checkpoint["optimizer_type"] = type(optimizer)
checkpoint["compression_sched"] = distiller.CompressionScheduler(net).state_dict() 


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainLoader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return train_loss/len(trainLoader), 100.*correct/total

best_acc = 0
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testLoader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        extras = {
             "current_top1" : acc, "best_top1" : acc, "best_epoch": epoch
        }
        checkpoint["extras"] = extras
        checkpoint["state_dict"] = net.state_dict(),
        checkpoint["optimizer_state_dict"] = optimizer.state_dict()
        checkpoint["epoch"] = epoch
        torch.save(checkpoint, '/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/models/' + name + '-' + str(epoch) + '-' + str(acc))
        best_acc = acc
    return acc


for epoch in range(epochs):
    startTime = time.time()
    trainLoss, trainAcc = train(epoch)
    testAcc = test(epoch)
    scheduler.step()
    endTime = time.time() - startTime
    print("Trn L: {:.4f}".format(trainLoss) + " " + "Trn A: {:.4f}".format(trainAcc) + " " + "Test A: {:.4f}".format(testAcc) + str(endTime // 60) + ":" + str(int(endTime % 60)))


Epoch: 0
Saving..
Trn L: 2.3633 Trn A: 12.6860 Test A: 19.03000.0:33

Epoch: 1
Saving..
Trn L: 2.0145 Trn A: 22.5060 Test A: 25.16000.0:32

Epoch: 2
Saving..
Trn L: 1.7856 Trn A: 31.4620 Test A: 34.81000.0:32

Epoch: 3
Saving..
Trn L: 1.6073 Trn A: 38.8800 Test A: 37.13000.0:32

Epoch: 4
Saving..
Trn L: 1.3270 Trn A: 51.4460 Test A: 48.71000.0:32

Epoch: 5
Saving..
Trn L: 1.0930 Trn A: 60.8680 Test A: 57.28000.0:32

Epoch: 6
Saving..
Trn L: 0.9361 Trn A: 66.8520 Test A: 65.11000.0:32

Epoch: 7
Trn L: 0.8312 Trn A: 70.9780 Test A: 60.23000.0:32

Epoch: 8
Saving..
Trn L: 0.7589 Trn A: 73.7560 Test A: 69.55000.0:32

Epoch: 9
Saving..
Trn L: 0.7040 Trn A: 75.7040 Test A: 76.23000.0:32

Epoch: 10
Trn L: 0.6590 Trn A: 77.2860 Test A: 71.64000.0:32

Epoch: 11
Saving..
Trn L: 0.6327 Trn A: 78.1420 Test A: 77.80000.0:32

Epoch: 12
Trn L: 0.5940 Trn A: 79.5860 Test A: 72.71000.0:32

Epoch: 13
Trn L: 0.5806 Trn A: 80.0500 Test A: 76.13000.0:32

Epoch: 14
Trn L: 0.5496 Trn A: 81.1660 Test A: 72.1